In [1]:
import pandas as pd
import time
import glob
import duckdb
import polars as pl

## Abriendo todos los json uno a uno

In [2]:
cur_time = time.time()
df1 = pd.read_json("Source Data/metadata-sitios/1.json",lines=True)
time_pandas1 = round((time.time() - cur_time),2)

In [3]:
cur_time = time.time()
df2 = pd.read_json("Source Data/metadata-sitios/2.json",lines=True)
time_pandas2 = round((time.time() - cur_time),2)

In [4]:
cur_time = time.time()
df3 = pd.read_json("Source Data/metadata-sitios/3.json",lines=True)
time_pandas3 = round((time.time() - cur_time),2)

In [5]:
cur_time = time.time()
df4 = pd.read_json("Source Data/metadata-sitios/4.json",lines=True)
time_pandas4 = round((time.time() - cur_time),2)

In [6]:
cur_time = time.time()
df5 = pd.read_json("Source Data/metadata-sitios/5.json",lines=True)
time_pandas5 = round((time.time() - cur_time),2)

In [7]:
cur_time = time.time()
df6 = pd.read_json("Source Data/metadata-sitios/6.json",lines=True)
time_pandas6 = round((time.time() - cur_time),2)

In [8]:
cur_time = time.time()
df7 = pd.read_json("Source Data/metadata-sitios/7.json",lines=True)
time_pandas7 = round((time.time() - cur_time),2)

In [9]:
cur_time = time.time()
df8 = pd.read_json("Source Data/metadata-sitios/8.json",lines=True)
time_pandas8 = round((time.time() - cur_time),2)

In [10]:
cur_time = time.time()
df9 = pd.read_json("Source Data/metadata-sitios/9.json",lines=True)
time_pandas9 = round((time.time() - cur_time),2)

In [11]:
cur_time = time.time()
df10 = pd.read_json("Source Data/metadata-sitios/10.json",lines=True)
time_pandas10 = round((time.time() - cur_time),2)

In [12]:
cur_time = time.time()
df11 = pd.read_json("Source Data/metadata-sitios/11.json",lines=True)
time_pandas11 = round((time.time() - cur_time),2)

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [16]:
def arreglo_cadenas(lista):
    for dfr in lista:
        dfr['hours'] = dfr['hours'].str.replace('\n', ' ')
        dfr['MISC'] = dfr['MISC'].str.replace('\n', ' ')

In [17]:
arreglo_cadenas([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [18]:
def cambia_tipos(lista):
    for dfr in lista:
        dfr['latitude']       = dfr['latitude'].astype('float32')
        dfr['longitude']      = dfr['longitude'].astype('float32')
        dfr['avg_rating']     = dfr['avg_rating'].astype('float32')
        dfr['num_of_reviews'] = dfr['num_of_reviews'].astype('int16')

In [19]:
cambia_tipos([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [20]:
def salva_parquet(lista):
    for i,dfr in enumerate(lista):
        dfr.to_parquet(f'parquet/source/{i+1}.parquet')

In [21]:
salva_parquet([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

## Leer con polars

In [2]:
#lazy
cur_time = time.time()
pl_df = (pl.scan_parquet('parquet/source/*.parquet').collect())
time_polars = round((time.time() - cur_time),2)
print(f"tiempo de lectura de todos los parquet usando polars en modo lazy = {time_polars} segundos")

tiempo de lectura de todos los parquet usando polars en modo lazy = 3.88 segundos


In [ ]:
dicc = {
    'tiempo':[time_pandas,time_pandas2,time_pandas3,time_pandas4,time_pandas5,time_pandas6,time_pandas7,time_pandas8,time_pandas9,time_pandas10,time_pandas11]
}
df_tiempo_pandas = pd.DataFrame(dicc,index=['time_pandas1','time_pandas2','time_pandas3','time_pandas4','time_pandas5',
                                              'time_pandas6','time_pandas7','time_pandas8','time_pandas9','time_pandas10','time_pandas11'])
print(f'Tiempo de lectura de los json en pandas = {round(df_tiempo_pandas.tiempo.sum()/60,2)} minutos')

In [5]:
pl_df.shape

(3025011, 15)

## Para salvar en formato csv tengo que convertir los tipos [] en strings

In [6]:
cur_time = time.time()
pl_df = (pl_df.lazy()
.with_columns([
    pl.col('category').apply(lambda x: ','.join(x)),
    pl.col('relative_results').apply(lambda x: ','.join(x)) 
]).collect()
)
print(f'Tiempo = {round((time.time() - cur_time),2)} segundos')

Tiempo = 85.36 segundos


In [7]:
pl_df.head()

name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
str,str,str,str,f32,f32,str,f32,i16,str,f64,f64,str,str,str
"""Porter Pharmac...","""Porter Pharmac...","""0x88f16e41928f...",null,32.388302,-83.357101,"""Pharmacy""",4.9,16,null,null,null,"""Open ⋅ Closes ...","""0x88f16e419294...","""https://www.go..."
"""City Textile""","""City Textile, ...","""0x80c2c98c0e3c...",null,34.01889,-118.215286,"""Textile export...",4.5,6,null,null,null,"""Open now""","""0x80c2c624136e...","""https://www.go..."
"""San Soo Dang""","""San Soo Dang, ...","""0x80c2c778e3b7...",null,34.05809,-118.29213,"""Korean restaur...",4.4,18,null,null,null,"""Open ⋅ Closes ...","""0x80c2c78249ab...","""https://www.go..."
"""Nova Fabrics""","""Nova Fabrics, ...","""0x80c2c89923b2...",null,34.02367,-118.232933,"""Fabric store""",3.3,6,null,null,null,"""Open ⋅ Closes ...","""0x80c2c8811477...","""https://www.go..."
"""Nobel Textile ...","""Nobel Textile ...","""0x80c2c632f933...",null,34.036694,-118.24942,"""Fabric store""",4.3,7,null,null,null,"""Open ⋅ Closes ...","""0x80c2c62c4960...","""https://www.go..."


In [8]:
pl_df.write_parquet('parquet/sitios1.parquet')
pl_df.write_csv('csv/sitios1.csv')

In [ ]:
pd_df = pl_df.to_pandas()